In [ ]:
import pandas as pd
import numpy as np
import pprint as pprint
import time


###Loadind data from csv files into datafreames


In [ ]:
# Restaurants
dfaccepts=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozaccepts.csv',encoding='ascii')
dfcuisine=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozcuisine.csv',encoding='utf8')
dfhours4=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozhours4.csv',encoding='utf8')
dfparkingdata2=pd.read_csv('Restaurant-data/chefmozparking.csv')
dfgeoplaces=pd.read_csv(filepath_or_buffer='Restaurant-data/geoplaces2.csv',encoding='latin-1')
# # consumers
dfusercuisine=pd.read_csv('Restaurant-data/usercuisine.csv')
dfuserpayment=pd.read_csv('Restaurant-data/userpayment.csv')
dfuserprofile=pd.read_csv('Restaurant-data/userprofile.csv')
# user item rating
dfrating=pd.read_csv('Restaurant-data/rating_final.csv')


###cleaning Data 

In [ ]:
# data cleaning
##Geopplaces
# replace all ? with nan
def cleandfgeoplaces():
    withoutqm=dfgeoplaces.replace('?',np.NaN)
#     nulvalues=withoutqm.isnull()
    withoutqm.isnull().sum(axis=0)
    # Drop null collunms bellow a given threshhod here 74. Removes zip,url and fax
    droppedCollumns=withoutqm.dropna(thresh=74,axis=1)
    clean=droppedCollumns.isnull().sum(axis=0)
    # droppedCollumns.isnull().sum(axis=1)
    
    #replace nul adresses with labels unnamed address1
  
    return clean

# cleandfgeoplaces()
def cleandfuserprofile():
    # find and replace all questionmarks with NAN 
    withoutqm=dfuserprofile.replace('?',np.NaN)
    clean=withoutqm.isnull().sum(axis=0)
    return dfuserprofile


def cleandf(data):
    data.replace('?',np.NaN,inplace=True)
    print('no. of duplicates', data.duplicated().sum(axis=0))
    print('null rows',data.isnull().sum(axis=0))
    data.drop_duplicates(inplace = True)
#     adult['sex'].map(lambda x: x.lower()).head()
    print(data.describe())
    return 0

import re

def cleanHours(x):
     result = re.split(';|-',x)
#     return result[0],result[1]

def dfhours4clean():
    res= dfhours4.hours.map(lambda x:cleanHours(x))
    return dfhours4.hours
#     check for and remove dublicates
# check and interpolate Nan values
# check on the dates that are erronious and resolve
# general cleaning()
#to use apply groupby to fill in collumns with mean vallues of the clossest collumns

# dfhours4clean()

# time.strptime("30 Nov 00", "%d %b %y")



In [ ]:
# 2.1.3. What are the names of different restaurants in the state of ‘tamaulipas’?
# select where name is tamaulipas
dfgeoplaces.loc[:,['name']][(dfgeoplaces.state=='tamaulipas')]


In [ ]:
# 2.1.4. How many different customers used public transport for going to therestaurants?
len(dfuserprofile[dfuserprofile['transport']=='public'])


In [ ]:
# 2.1.5. What is the least popular payment method among customers?
min(dfuserpayment.groupby('Upayment'))


In [ ]:
# 2.1.6. How many (different) restaurants work until 19:00 in the evenings?
# len(dfhours4[dfhours4=='00:00-19:00'])
dfhours4.hours
# pd.to_timedelta(dfhours4.hours)

In [ ]:
# 2.1.7. Which type of food (rcuisine) is served most among the restaurants?
dfcuisine[['Rcuisine']].mode()


In [ ]:
# 2.1.8. What is the percentage of customers who were born between 1980 and 1990?
len(dfuserprofile[(dfuserprofile.birth_year>=1980) & (dfuserprofile.birth_year<=1990)])/len(dfuserprofile)*100

In [ ]:
# 2.1.9. What is the percentage of students with a medium budget preferring walking to the restaurants?
len(dfuserprofile[(dfuserprofile.budget=='medium') &(dfuserprofile.transport=='on foot')])/len(dfuserprofile)*100


###Assignment2- Part 2:

In [ ]:
# 2.2.1. What are the names of restaurants that do not have public parking lots?
dfgeoplaces[['placeID','name']].merge(dfparkingdata2[dfparkingdata2.parking_lot!='public'],how='inner',on='placeID').name

In [ ]:
# 2.2.2. What are the addresses of restaurants which only accept ‘cash’?
dfgeoplaces.merge(dfaccepts[dfaccepts.Rpayment=='cash'],on='placeID',how='inner').address


In [ ]:
# 2.2.3. Name the cities where the restaurants cook and serve ‘fast food’?

dfgeoplaces[['placeID','city']].merge(dfcuisine[dfcuisine.Rcuisine=='Fast_Food'],how='inner',on='placeID').city ##names not distict=== to add distinct


In [ ]:
# 2.2.4. What is the most common ‘rating’ among customers with family?

dfrating[['userID','rating']].merge(dfuserprofile[dfuserprofile.ambience=='family'],on='userID',how='inner').rating.mode()


In [ ]:
# 2.2.5. What types of food (‘rcuisine’) received the highest rank from people with‘low’ budget?

#filter each dataset by conditions and merging the filtered datasets 
food=dfcuisine.merge(dfrating[['userID','placeID','food_rating']],on='placeID',how='left').merge(dfuserprofile[['userID','budget']][dfuserprofile.budget=='low'],on='userID',how='inner')
#grouping the merged output by rquisin and getting sum for each cuisine
groupedSum=food.groupby('Rcuisine').sum()
#get the maximum Id value based on the collum foodrating
groupedSum.food_rating.idxmax()


In [ ]:
# 2.2.6. What is the average of ‘service rating’, received from ‘social drinkers’ about restaurants which just served ‘Wine-Beer’?


label=dfrating[['userID','placeID','service_rating']].merge(dfuserprofile[['drink_level','userID']][dfuserprofile.drink_level=='social drinker'],on='userID',how='inner')
label2=label.merge(dfgeoplaces[['placeID']][dfgeoplaces.alcohol=='Wine-Beer'],on='placeID',how='inner')
label2.service_rating.mean(axis=0)


In [ ]:
# 2.2.7. How many smokers gave zero ‘service rating’ to the restaurants without an open area?
len(dfuserprofile[dfuserprofile.smoker=='true'].merge(dfrating[dfrating.service_rating==0], on='userID',how='inner').merge(dfgeoplaces[dfgeoplaces.area!='open'],on='placeID',how='inner'))
#=================================

In [ ]:
# 2.2.8. Find the correlation between different rating categories ((general) rating, food_rating, service_rating) with the price levels of the restaurants.
# Hint: To answer the last question, you need to change the type of attribute values
# from categorical to numerical.
dfrating.corr
dfgeoplaces.price.map(lambda x: 1 if x == 'low' else 2 if x=='medium' else 3 ) # let 1=low ,2=medium,3=high
dfrating.rating.corr(dfrating.food_rating)
dfrating.rating.corr(dfrating.service_rating)
